# Notebook para Análisis Matemático 2

La idea es ir agregándole herramientas a medida que se vayan aprendiendo. Más que nada es para ahorrarte o corroborar algunas cuentas.

Primero importamos las librerías que vamos a usar. La idea de esto es decirle a python que se tenga a mano algunas herramientas:

* SymPy: Es la que nos deja hacer cálculo simbólico. Con simbolitos.
* Matplotlib: Es la que nos deja hacer gráficos. Llamamos en particular a pyplot porque es la que se usar y la más usada en general.
* NumPy: Es una que te permite hacer cuentas grandes de manera súper optimizada. Clave para cálculo numérico.

In [4]:
import sympy as smp
import matplotlib.pyplot as plt
import numpy as np

## Extremos con restricciones 

Lo primero que hacemos es definir las cosas correctamente según Sympy

* Las variables las llamamos en una línea y las funciones en otra. 
* Las últimas tienen que tener el argumento cls=smp.Function. 
* Le decimos las variables de las que dependen.

Si querés podés agregar más funciones o variables acordándote de seguir el formato anterior.

In [65]:
# Esto es un comentario. No se corre lo que le sigue al numeral y no afecta en nada al resto.
x,y,l = smp.symbols(r'x y \lambda', real=True)  # Fijate que puse r' ...' eso es para que me tome la letra griega lambda.
f, g = smp.symbols('f g', real=True, cls=smp.Function)
f = f(x,y); g=g(x,y)

Acá definí las funciones como te parezca. En esta notebook f es la función a estudiar y g el vínculo. Si querés otro vínculo acordate de declararlo en la celda anterior y después venís acá.

In [66]:
f = 2*x**2 + y**4
g = x**2 + 2*y**2 - 8

In [67]:
# Si escribís el nombre de una variable en la última línea de la celda, Jupyter te muestra
# que hay en esa variable. En este caso tiene a la función simbólica g definida como en la 
# celda anterior.
g

x**2 + 2*y**2 - 8

### Gradiente y extremos locales

En la siguiente celda calculamos el gradiente de las funciones.

* smp.diff() es la función que hace el trabajo.
* Si tu función depende de más variables fijate de cambiar eso acá también. El único cambio que habría que hacer es agregarle las variables nuevas a la lista "var".
* Notación de Python: las cosas que ponemos entre corchetes se llaman listas. Ej: lista_de_ej = [1, 2, 3]

In [8]:
# Si en vez de x,y tengo una función que depende de x,y,z; la línea de abajo debería decir
# var = [x,y,z]
var = [x,y]

# Las líneas que siguen quedarían igual. Si tenés una restricción más, podés definirle e
# gradiente de la misma manera. Ej: si tenés una función de vínculo h, escribís:
# grad_h = [smp.diff(h, u) for u in var]

grad_f = [smp.diff(f, u) for u in var]
grad_g = [smp.diff(g, u) for u in var]
grad_f

[4*x, 4*y**3]

In [70]:
candidatos_locales = smp.solve(grad_f, (x,y))
candidatos_locales

[(0, 0)]

### Hessiana

Es la misma idea que el gradiente solo que ahora derivás al gradiente que calculaste antes.

* En el caso de tener 3 variables en vez de 2, lo único que tenés que cambiar es el límite superior de la variable i (se explica mejor en la celda)
* smp.Matrix(): a esta función le das de comer una lista y te devuelve una matriz con el formato lindo ese que se ve.
    * Además es necesaria porque para el método de Sylvester te interesa calcularle el determinante después.
    * También podés usar el método de los autovectores y autovalores muy rápido con el método .eigenvects()

Si te pinta resolver como calcular los menores con Python podés mirar las herramientas que hay para matrices acá https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html

In [78]:
# Si en vez de 2, tenés n variables, donde dice for i in [0,1], escribí: for i in [0, n-1]
hess_f = smp.Matrix([[smp.Rational(1,2)*smp.diff(grad_f[i], u) for u in var] for i in [0,1]])
hess_f

Matrix([
[2,      0],
[0, 6*y**2]])

#### Criterio de Sylvester

Para el caso de una función real de dominio $\mathbb{R}^2$ y diferenciable en $(x_0,y_0)$, este criterio nos dice que
* Si $H_{11}(x_0,y_0) > 0$ y $|H(x_0,y_0)|>0$ entonces $f$ tiene un mínimo local en (x_0,y_0).
* Si $H_{11}(x_0,y_0) > 0$ y $|H(x_0,y_0)|<0$ entonces $f$ tiene un mínimo local en (x_0,y_0).
* Si $H_{11}(x_0,y_0) \le 0$ o $|H(x_0,y_0)|=0$ entonoces el criterio no decide nada.

In [102]:
subs = [[(var[i], candidatos_locales[j][i]) for i in range(len(var))] for j in range(len(candidatos_locales))]
H_11_eval_en_cand = [hess_f[0].subs(subs[i]) for i in range(len(subs))]
det_H_eval_en_cand = [hess_f.det().subs(subs[i]) for i in range(len(subs))]
testarudos = []
max_loc, min_loc = [], []

for i in range(len(candidatos_locales)):
    if (H_11_eval_en_cand[i]*det_H_eval_en_cand[i] == 0 or H_11_eval_en_cand[i]<0):
        print('Acá no se qué decirte :(')
        testarudos.append(candidatos_locales[i])
        continue
    elif (det_H_eval_en_cand[i]>0):
        min_loc.append(candidatos_locales[i])
    else:
        max_loc.append(candidatos_locales[i])

Acá no se qué decirte :(


#### Criterio de los autovalores (este no falla) COMPLETAR

In [103]:
# .eigenvects() te devuelve una lista de tuplas de la forma
# (autovector, multiplicidad algebraica, lista de autovectores)
# También existe el método .eigenvals() que te devuelve una lista de autovalores.
# Calcular los autovectores le cuesta más a la compu, pero eso empieza a ser relevante 
# para matrices mucho más grandes de las que vamos a tratar acá. Solo lo comento como
# curiosidad.
hess_f.eigenvects()

[(2,
  1,
  [Matrix([
   [1],
   [0]])]),
 (6*y**2,
  1,
  [Matrix([
   [0],
   [1]])])]

## Multiplicadores de Lagrange

Hasta acá sabemos que tenemos una función f con una restricción g. Queremos resolver el sistema

$$
\begin{cases}
\nabla f - \lambda \nabla g = 0 \\
g = 0
\end{cases}
$$

Para eso el plan de acción es armar el sistema de ecuaciones como una lista, donde cada elemento es una de las ecuaciones. Es clave tener en cuenta que los resolvedores solo manejan ecuaciones homogéneas, por eso definí al sistema y a la restricción de esa manera.

In [12]:
# Armamos el sistema de ecuaciones.
# Si en vez de dos variables tenés n, acordate de hacer el cambio de [0,1] a [0, n-1] en la línea que sigue.

lagrange = [grad_f[i] - l*grad_g[i] for i in [0,1]]  # Primeras dos ecuaciones.
lagrange.append(g)                                   # Le agrego a la lista el vínculo.
lagrange                                             # Mostramos el sistema completo.

[-2*\lambda*x + 4*x, -4*\lambda*y + 4*y**3, x**2 + 2*y**2 - 8]

El resolvedor que vamos a usar es smp.solve(). 

Recibe como parámetros:

1) El sistema de ecuaciones (homogéneo) a resolver. Es una lista. Capaz también una matriz, no me acuerdo.
2) Una tupla (encierra elementos entre corchetes) con las variables a despejar.
    * Fijate que le paso a $\lambda$ como variable también para que me diga que valor toma. Importante esto.
    * En caso de tener más variables tu sistema de ecuaciones (variable lagrange definida en la celda anterior) debería tener más ecuaciones. El único cambio que habría que hacer en esta celda es agregarle las variables nuevas a la tupla de variables.
    
Te devuelve:

* Una lista con tuplas de valores que resuelven el sistema.
* Las tuplas están ordenadas. Es decir van a tener el mismo orden que vos le pasaste a la función como argumento.

In [13]:
# Vamos a guardar la lista de tuplas solución en una variable que llamamos
# lagr_sols ¿Por qué? porque se puede y porque capaz que más adelante las quiero
# usar para algo. Me parece una buena práctica guardarlas así.
lagr_sols = smp.solve(lagrange, (x,y,l))
lagr_sols

[(-2, -sqrt(2), 2),
 (-2, sqrt(2), 2),
 (0, -2, 4),
 (0, 2, 4),
 (2, -sqrt(2), 2),
 (2, sqrt(2), 2),
 (-2*sqrt(2), 0, 2),
 (2*sqrt(2), 0, 2)]

### Evaluar en los candidatos para obtener los extremos

In [64]:
# Acá voy a preparar la lista de tuplas que necesito para hacer la evaluación de f en estos puntos.

sustituciones = [[(var[i], lagr_sols[j][i]) for i in range(len(var))] for j in range(len(lagr_sols))]
sustituciones

candidatos = [f.subs(sustituciones[i]) for i in range(len(lagr_sols))]  # Hago una lista con las imágenes de los candidatos a extremo
minimo, maximo = min(candidatos), max(candidatos)                       # Me fijo cuál es el valor máximo y mínimo

# Acá hago dos listas, donde cada elemento contiene una lista con los extremos y su imágen.

minimos, maximos = [], []
for i in range(len(candidatos)):
    if (candidatos[i] == minimo):
        minimos.append([sustituciones[i], candidatos[i]])
    elif (candidatos[i] == maximo):
        maximos.append([sustituciones[i], candidatos[i]])
        
minimos

[[[(x, -2), (y, -sqrt(2))], 12],
 [[(x, -2), (y, sqrt(2))], 12],
 [[(x, 2), (y, -sqrt(2))], 12],
 [[(x, 2), (y, sqrt(2))], 12]]